In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("proj01.ipynb")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

# Project 1: Regional GDP
### Regional Heterogeneity: Varying Economic Performance in US Counties
In the first project of Econ 148, we will examine county-level economic performance as measured by real Gross Domestic Product (GDP) over the past two decades. As the most important macroeconomic indicator, GDP is often analyzed at the country level; however, regional heterogeneity is also a crucial source of variation in growth and business cycle analysis. Therefore, in this project we will use a county-level real GDP dataset from the Bureau of Economic Analysis (BEA) and try to find out the regional differences in economic performance, especially during recessions.

You will use the data cleaning and data manipulation skills you have learned so far in this course to wrangle this rich, but rather complex, real-world dataset. 

#### Data sources: 

The main dataset we will use in this notebook is ["CAGDP9: Real GDP in Chained Dollars by County and MSA"](https://www.bea.gov/data/gdp/gdp-county-metro-and-other-areas) from the Bureau of Economic Analysis (BEA). It provides a comprehensive measure of the gross domestic product of counties, metropolitan statistical areas, and some other local areas in the United States from 2001 to the present. Accessed Jan 2023. **We use a subset of the full dataset (about 50%) that includes some of the industries available in the original dataset.**

We will also use the ["United States Counties Database"](https://simplemaps.com/data/us-counties) from Simplemaps.com. Specifically, we will use the geographic data of U.S. counties (i.e., latitude and longitude) to create the visualizations in the last section. Accessed Jun 2022. 

**Learning Objectives:**   
- Importing and exporting dataframes
- Metadata of a dataframe
- Recognizing and handling missing values and NaNs
- String methods and type conversions
- Grouping and aggregating
- Calculating changes and percentage changes
- Joining and merging two dataframes
- A demo of using Jupyter widgets

**A Note on Grading:**  
In Project 1, the autograded questions will have hidden tests, and the text-based free response questions will be graded on correctness. 

---
## Part 1: Importing dataset

Datasets are encoded in different codecs. In most cases, the default codex (utf-8) will be able to process the datasets. But in other cases, if we run into some issues with decoding (especially with datasets containing symbols or other languages), we can manually specify other codecs (e.g. ascii, latin-1). A complete list of codecs for Python 3.7 and newer can be found [here](https://docs.python.org/3.7/library/codecs.html#standard-encodings). 

As a side note, if we want to export the dataframe when we are done, we will also want to make sure that we are using the correct codecs.

We have got an immediate example here. **Some of you** may not be able to import the real GDP dataset by BEA with the default codecs (utf-8). 
```python
>>> rgdp = pd.read_csv("data/sample_CAGDP9__ALL_AREAS_2001_2021.csv")
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf1 in position 137852: invalid continuation byte
```

**Question 1.1:** Import the dataset `data/sample_CAGDP9__ALL_AREAS_2001_2021.csv` using an alternative codec `latin-1`.

Hint: Look up the [documentation](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) for `pd.read_csv` and see if there is any argument related to codec or encoding. 

Note: It's totally fine if you see a warning after you successfully import the dataset. This has to do with the content of this dataset.

In [ ]:
rgdp_raw = ...
rgdp_raw

In [ ]:
grader.check("q1_1")

**Question 1.2:** We notice that the last four rows in `rgdp_raw` are just some footnotes, so we will drop them. To do so, we can either select the top 47666 rows for the data that we want, or we can drop the bottom 4 rows with [`pandas.DataFrame.drop`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html). Assign the modified dataframe to `rdgp`. 

In [ ]:
rgdp = ...
rgdp

In [ ]:
grader.check("q1_2")

---
## Part 2: Learn about the dataset

Like we did in Lab 3, one of the first things that we will do with our dataset is to learn about its structure: how many rows and columns are there in the dataset? What values does each column store? What is the data type for each column (int, string, etc.)? For categorical variables, what are unique values? For numerical variables, what is the mean, median, min, and max? 

In this section, we will use the built-in functions in Pandas to quickly answer the question above. 

**Question 2.1:** How many rows and columns are there in this dataframe `rdgp`? Assign the number of rows to `N_rows` and the number of columns to `N_cols`. 

Hint: The first section of lab 3 can be a good reference. 

In [ ]:
N_rows = ...
N_cols = ...
N_rows, N_cols

In [ ]:
grader.check("q2_1")

**Question 2.2:** How many unique counties are there in this dataframe `rdgp`? Assign the number of unique counties to `N_unique_counties`. 

In [ ]:
N_unique_counties = ...
N_unique_counties

In [ ]:
grader.check("q2_2")

**Question 2.3:** What do the values in the "Description" column represent? Are the categories in the "Description" column mutually exclusive, or are they potentially subsets of each other? Give an example to illustrate your point. 

_Type your answer here, replacing this text._

**Question 2.4:** What are the data types of columns `GeoFIPS`, `GeoName`, `Unit`, and `2021`? Are they integers, floats, strings, objects, or mixed types? Do you find any data types in these columns problematic? For example, they should be integers, but are instead coded as strings. 

Hint: you can find the data type by printing out a series (e.g. `rdgp["GeoFIPS"]`) and then look at the last line in the output. There you can see the `dtype` of this column. Or you can checkout the data types for all the columns in the dataframe using `df.dtypes`. 

_Type your answer here, replacing this text._

### (Optional) Exploratory Data Anaylsis with pandas-profiling

In addition, we can also rely on other wonderful python packages to get a big picture of our dataset. As an example, the [pandas-profiling](https://pandas-profiling.ydata.ai/docs/master/index.html) package "generates profile reports from a pandas DataFrame. The pandas `df.describe()` function is handy yet a little basic for exploratory data analysis. pandas-profiling extends pandas DataFrame with `df.profile_report()`, which automatically generates a standardized univariate and multivariate report for data understanding." 

In a way, pandas-profiling can do most of the exploratory data analysis tasks and create a comprehensive and beautiful report for you within a minute. 

Copy the following code to a code cell and run it to see the report generated by pandas-profiling if you have the package installed. 

```python
from pandas_profiling import ProfileReport
profile = ProfileReport(rgdp, title="Pandas Profiling Report")
profile.to_notebook_iframe()
```

---
## Part 3: Missing Values and NaNs

The difference between the data found in many tutorials and real-world data is that real-world data is rarely clean and homogeneous. In particular, many interesting datasets will have some amount of missing data. To complicate matters, different data sources may report missing data in different ways.

In our dataset, there are two types of "missing values": (D) and (NA). 

In [ ]:
rgdp[rgdp["2001"] == "(D)"][:5]

In [ ]:
rgdp[rgdp["2001"] == "(NA)"][:5]

**Question 3.1:** Look up the [footnote](https://www.econ148.org/sp23/resources/assets/supp_materials/proj01/CAGDP9__Footnotes.html) of this dataset, what does each of these two types of missing values represent? What do you think is a good way to handle these two types of missing values respectively? This is an open-ended question. 

_Type your answer here, replacing this text._

**Question 3.2:** We will not develop a strategy for replacing missing values in this project. For our purposes, simply drop all rows that contain missing values (either (D) or (NA)). 

Hint: One way to do this is to first replace all instances of (D) and (NA) with `np.nan` and then use `dropna` on the dataframe. [`pandas.DataFrame.replace`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html) may be helpful in this case. But there are many other ways to do this, so free feel to use whichever method that makes the most sense to you. 

In [ ]:
rgdp_no_nans = rgdp.copy()
# drop rows with missing values
...
rgdp_no_nans = ...
rgdp_no_nans.head()

In [ ]:
grader.check("q3_2")

---
## Part 4: Type conversions and string methods

In part 2, we noticed that the dataframe we are working with does not have the desired data types for many columns. For example, the real GDP data has some entries that are kept as strings. To convert these entries to the desired data types, the most common way is to use [`pandas.DataFrame.astype(type)`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.astype.html). 

In [ ]:
# make a copy of the dataframe first
rgdp_clean = rgdp_no_nans.copy()

For example, we can convert the column "2001" from strings to integers using `astype("int64")`.

In [ ]:
rgdp_no_nans["2001"]

In [ ]:
rgdp_no_nans["2001"].astype("int64")

**Question 4.1:** Write a for loop that converts all values in all columns from "2001" to "2021" in `rgdp_clean` to `int64`. 

Hint: the column labels (i.e. "2001", "2002", etc.) are strings. So when you access the columns, you may want to convert to the years to strings first using [`str(...)`](https://www.w3schools.com/python/ref_func_str.asp). 

In [ ]:
for year in ...:
    rgdp_clean[str(year)] = ...

In [ ]:
grader.check("q4_1")

In [ ]:
# check to see if the data types are correct now
rgdp_clean.dtypes

Now we can see the `dtype` for 2001-2021 are `int64`.  

We can also convert data type into `float`, `str`, etc. using the `astype` method on the entire dataframe or some specified data series. Pandas also provides a [`pandas.to_numeric()`](https://pandas.pydata.org/docs/reference/api/pandas.to_numeric.html) function to easier convert different data types into numeric. 

In [ ]:
rgdp_clean["LineCode"] = rgdp_clean["LineCode"].astype("int64")

### Regular Expressions and Strings

But sometimes the data entries require some manipulation before can be converted to the desirable data types easily. For example, entries in the `GeoFIPS` column in our dataset has the following form. 

In [ ]:
rgdp_no_nans["GeoFIPS"]

Note that the parentheses are there in the data, so simple conversion like `astype(int)` will fail.

In [ ]:
# this will produce an error
# rgdp_no_nans["GeoFIPS"].astype("int64")

To extract relevant information, we will use regular expressions. A regular expression is a sequence of characters that specifies a search pattern in text. Usually such patterns are used by string-searching algorithms for "find" or "find and replace" operations on strings, or for input validation.

For example, we can extract student ID among a bunch of other texts. 

In [ ]:
import re
some_text = "Name: Oski, Age: 999, SID: 12345678"
re.findall("SID: (\d*)", some_text)

Or we can replace some text we want. 

In [ ]:
some_other_text = "Stanford is the No.1 university in California. "
re.sub("Stanford", "Berkeley", some_other_text)

Note that a common way to get rid of texts in a specified pattern is to use [`re.sub`](https://docs.python.org/3/library/re.html#re.sub) and replace the pattern with the empty strings. For example: 

In [ ]:
some_messy_text = "Beroskikeleoskiy oskiis oskitheoski No.1oski univoskiersioskity ioskin Calioskifooskirnia."
re.sub("oski", "", some_messy_text) # substitute with the empty string

Regular expression is a deep topic and it requires practice to be able to use it well. A well-known website to test if your regular expression works or not is [regex101](https://regex101.com/). It will be very helpful skill in terms of data cleaning. But for now, we will just use it to get rid of the parentheses in data entries in the `GeoFIPS` column. 

**Question 4.2:** Remove all instances of quotation marks from `too_many_quotation_marks` using regular expressions. 

In [ ]:
too_many_quotation_marks = 'B"er"kel"ey i"s t"he" No".1 u"niv"ersity i"n C"ali"fo"rn"i"a.'
no_quotation_marks = ...
no_quotation_marks

In [ ]:
grader.check("q4_2")

To apply regex and many other string method to a dataframe, we can use `pandas.Series.str` methods, and apply a string function. In our case [`pandas.Series.str.replace`](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.replace.html) that can replace each occurrence of pattern/regex in the Series/Index. 


**Question 4.3:** Write the code below that first delete all the quotation marks in values in the `GeoFIPS` column in `rgdp_clean` with [`pandas.Series.str.replace`](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.replace.html) and then convert all strings into integers using `astype('int64')`.

In [ ]:
...
rgdp_clean

In [ ]:
grader.check("q4_3")

In [ ]:
rgdp_clean.dtypes

Now everything is in the correct data type.

---
## Part 5: Pivot tables and melt

Pivot table and melt are inverses of each other. 

Many economic datasets are in "spreadsheet" formats, which have groups of columns representing the same type of information. For example, in our real GDP dataframe, columns like "2001", "2002" simply give the real GDP values in the given year. To make our lives easier when working with the data later, we can convert the dataframe to a more traditional dataframe, where all data are in just one column. So, [`pandas.melt`](https://pandas.pydata.org/docs/reference/api/pandas.melt.html) gives us a way to unpivot a DataFrame from wide to long format.

**Question 5.1:** Convert the dataframe using [`pandas.melt`](https://pandas.pydata.org/docs/reference/api/pandas.melt.html) so that it contains seven columns: `GeoFIPS`, `GeoName`, `Region`, `LineCode`, `Description`, `year`, and `value`. The `value` is the real GDP value for the given region, industry, and year on that row.

In [ ]:
rgdp_melted = pd.melt(rgdp_clean, 
                      id_vars=[...], 
                      value_vars=[...]
                     ).rename(columns={"variable": "year"})
rgdp_melted

In [ ]:
grader.check("q5_1")

Now, all the real GDP values are in just one column. 

**Question 5.2:** One issue remains: the `year` column has data type as strings. Convert the column into `int64` using your favorite method. 

In [ ]:
...
rgdp_melted["year"]

In [ ]:
grader.check("q5_2")

Before we start with the next section, since we will only concern about **county-level** GDP data of **all industries** for now, we will filter the dataframe for only the relevant data. **Some entries in the dataframe represent state aggregate or national aggregate, and these entries will have GeoFIPS ending in 000.** For example, 00000 represents the entire US; 01000 represents the Alabama state; 01001 represents Autauga--a county in Alabma. 

**Question 5.3:** Filter the `rdgp_melted` dataframe for rows that contain real GDP data for **only counties (not aggregates)**. Assign the filtered dataframe to `rgdp_county`. So in the end `rgdp_county` dataframe should have seven columns: `GeoFIPS`, `GeoName`, `Region`, `LineCode`, `Description`, `year`, and `value`. 

Hint: You can get the remainder of a division using the modulo operator % in Python. 

In [ ]:
rgdp_county = ... # filter for relevant rows

rgdp_county = rgdp_county.drop(...) # drop `Description` column
rgdp_county["Region"] = rgdp_county["Region"].astype(int) # we can finally do this!
rgdp_county

In [ ]:
grader.check("q5_3")

**Question 5.4** Now filter the `rgdp_county` dataframe for rows that contain **all industry total** real GDP data for only counties (not aggregates). Assign the filtered dataframe to `rgdp_county_allindustry`, then drop the column `Description`. So in the end `rgdp_county_allindustry` dataframe should have six columns: `GeoFIPS`, `GeoName`, `Region`, `LineCode`, `year`, and `value`. 

In [ ]:
rgdp_county_allindustry = ...
rgdp_county_allindustry

In [ ]:
grader.check("q5_4")

Now we have our dataframe consisting of county-level real GDP data of all industries total. 

---
## Part 6: Groupby and aggregate functions

Pandas groupby is used for grouping the data according to the categories and apply a function to the categories. It also helps to aggregate data efficiently. Basically, with Pandas groupby, we can split Pandas data frame into smaller groups using one or more variables. After splitting, we can apply aggregating functions that reduce the dimension of the grouped object and give us the result that we want. Pandas offers several built-in aggregating functions as we have discussed in lectures and discussion sections. 

As examples, now we will work with `groupby` and two aggregating functions to calculate mean and median. 

**Question 6.1:** Find the annual average GDP for all indutries in all US counties. And assign the result to `rgdp_county_allindustry_mean`. This dataframe should only have two columns: `year` and `value`. 

Hint: `pandas.Dataframe.groupby` may be helpful. 

In [ ]:
rgdp_county_allindustry_mean = ...
rgdp_county_allindustry_mean.head()

In [ ]:
grader.check("q6_1")

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(rgdp_county_allindustry_mean["year"], rgdp_county_allindustry_mean["value"])
plt.xticks(np.arange(2001, 2022, 3))
plt.xlabel("Year")
plt.ylabel("Thousands of chained 2012 dollars")
plt.title("Mean Real GDP for All Industries in U.S. Counties (2001-2021)");

**Question 6.2:** Find the annual median GDP for all indutries in all US counties. And assign the result to `rgdp_county_allindustry_median`. This dataframe should only have two columns: `year` and `value`. 

Hint: `pandas.Dataframe.groupby` may be helpful. 

In [ ]:
rgdp_county_allindustry_median = ...
rgdp_county_allindustry_median.head()

In [ ]:
grader.check("q6_2")

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(rgdp_county_allindustry_median["year"], rgdp_county_allindustry_median["value"])
plt.xticks(np.arange(2001, 2022, 3))
plt.xlabel("Year")
plt.ylabel("Thousands of chained 2012 dollars")
plt.title("Median Real GDP for All Industries in U.S. Counties (2001-2021)");

**Question 6.3:** Compare and contrast the annual mean and median real GDP for all US counties. What do they share in common? What differences do they have? Why do you think this is the case? 

_Type your answer here, replacing this text._

---
## Part 7: Changes and percent changes

Analyzing raw changes and percent changes of economic data is pertinent to many economic research and studies. Pandas also provides convenience methods for us to obtain raws changes and percent changes between rows of the dataframe easily. 

In this part, we will use [`pd.DataFrame.diff`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.diff.html) and [`pd.DataFrame.pct_change`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pct_change.html) methods to see how median county level real GDP has changed in each region in U.S. during the past 20 years. Bureau of Economic Analysis categorizes all states into eight big regions, and the eight regions are shown below. 

<img src="assets/BEA_regions_iowa_state.jpg" width="600">
</br>
<center>U.S. Bureau of Economic Analysis Regions Reference Map</center>
<center>Source: <a href="https://www.icip.iastate.edu/maps/refmaps/bea">Iowa State University</a></center>

In [ ]:
bea_regions = {1: "New England", 
               2: "Mideast", 
               3: "Great Lakes", 
               4: "Plains", 
               5: "Southeast", 
               6: "Southwest", 
               7: "Rocky Mountains", 
               8: "Far West"}

**Question 7.1:** As an example, let's write the code that computes the **percent changes of median real GDP in the Far West region (coded as 8 in the dataset)** using [`pd.DataFrame.pct_change`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pct_change.html). Assign the result to `rgdp_far_west_pct_chg` which have two columns `year` and `value` (that stores the percent changes). 

Note: For example, in `rgdp_far_west_pct_chg`, a row with year 2002 should have a value on the same row that corresponds to the percent change in real GDP from 2001 to 2002. `rgdp_far_west_pct_chg` should not have any rows with NaN values. 

Hint: you may want to use `groupby`. 

In [ ]:
rgdp_far_west = rgdp_county_allindustry[...] # select relevant rows
rgdp_far_west = ... # find the median using groupby
rgdp_far_west= rgdp_far_west.reset_index()
rgdp_far_west_pct_chg["value"] = ... # find the percent changes
rgdp_far_west_pct_chg = ... # drop NaN values
rgdp_far_west_pct_chg

In [ ]:
grader.check("q7_1")

Here all columns are converted to percent changes. The column we are concerned about is the "value" column. Note that they are not in percentages (i.e. 0.01 represents 1%, not 0.01%), so if we want them in percentages, we will have to multiply the column by 100. 

Now we will make a plot for all regions. 

<!-- BEGIN QUESTION -->

**Question 7.2:** Now, we want to find the percent changes of median real GDP in each region with `pd.DataFrame.pct_change`. Similarly, let's write the code that computes the percent changes of median real GDP in each region. The dataframe `rgdp_region_pct_chg` which have two columns `year` and `value` (that stores the percent changes). 

Hint: start by copying your code from question 7.1.

In [ ]:
plt.figure(figsize=(8, 6))
for region in np.sort(rgdp_county_allindustry["Region"].unique()):
    rgdp_region = rgdp_county_allindustry[...] # select relevant rows
    rgdp_region = ... # find the median using groupby
    rgdp_region= rgdp_region.reset_index()
    rgdp_region_pct_chg["value"] = ... # find the percent changes
    rgdp_region_pct_chg = ... # drop NaN values
    plt.plot(rgdp_region_pct_chg["year"], 
             rgdp_region_pct_chg["value"] * 100, # as percentages
             label=bea_regions[region]
            )
    
plt.xticks(np.arange(2001, 2022, 3))
plt.xlabel("Year")
plt.ylabel("Percent")
plt.title("Percentage Changes of Median Real GDP for All Industries in U.S. Counties (2001-2021)")
plt.legend(title="Region", loc=(1.03, 0.58));

<!-- END QUESTION -->



---
## Part 8: Merge

To combine data on common columns or indices, the first technique that we will learn is `merge`. We can use `merge` anytime we want to join two dataframe that has one or more common columns. With `merge`, we can combine dataframe based on one or more keys, similar to what we would do in a relational database. 

In this section, we will combine our real GDP dataframe with another dataframe that contains the geographical data of each county, so that we can make some beautiful and informative visualizations in the next section. 

First, we will import the new dataset. 

In [ ]:
county_geo = pd.read_csv("data/uscounties_geo.csv")
county_geo

**Question 8.1:** Select only `county_fips`, `lat`, `lng`, `population` columns in the `county_geo` dataframe. 

In [ ]:
county_geo = ...
county_geo

In [ ]:
grader.check("q8_1")

Notice that the new dataset shares a common column "county_fips" with our real GDP dataframe "GeoFIPS". ([What is GeoFIPS?](https://en.wikipedia.org/wiki/FIPS_county_code)) So we will merge the two dataframe based on this key. 

**Question 8.2:** Merge `rgdp_county_allindustry` with `county_geo` on GeoFIPS. No need to drop the duplicate GeoFIPS column. 

In [ ]:
rgdp_county_allindustry_geo = ...
rgdp_county_allindustry_geo

In [ ]:
grader.check("q8_2")

---
## Part 9: Visualize the regional GDP!

Now we have cleaned up our dataset and computed the percent change of GDP for each county. It's time to use this data to show how economic performance vary across different regions! 

In [ ]:
rgdp_county_allindustry_geo_chg = rgdp_county_allindustry_geo[rgdp_county_allindustry_geo["year"] != 2001].copy()
rgdp_county_allindustry_geo_chg["value"] = rgdp_county_allindustry_geo[["value"]].pct_change()
rgdp_county_allindustry_geo_chg

To eliminate some outliers--some have extreme values for changes between years--we will only work with the data with GDP changes from 10th percentile to 90th percentile. The following function will plot the changes for a given year and a given industry. 

In [ ]:
def plot_counties(data, year, industry="All Industries"):
    
    MIN = np.nanpercentile(data["value"], 10)
    MAX = np.nanpercentile(data["value"], 90)
    
    rgdp_county_year = data[data["year"] == year] 
    rgdp_county_year = rgdp_county_year[(rgdp_county_year["value"] < MAX) & (rgdp_county_year["value"] > MIN)]
    
    plt.figure(figsize=(16, 9))
    plt.xlim([-170, -50])
    plt.ylim([10, 70])
    plt.scatter(rgdp_county_year["lng"], rgdp_county_year["lat"], 
                c=rgdp_county_year["value"], cmap="coolwarm", s=20, alpha=0.8)
    plt.colorbar()
    plt.title(f"Percentage Changes of Real GDP for {industry} in U.S. Counties ({year})", size=14)
    plt.tight_layout();

In [ ]:
plot_counties(rgdp_county_allindustry_geo_chg, 2002)

In [ ]:
plot_counties(rgdp_county_allindustry_geo_chg, 2008)

In [ ]:
plot_counties(rgdp_county_allindustry_geo_chg, 2020)

<!-- BEGIN QUESTION -->

**Question 9.1:** Comment on the results above. Are the economic performance similar or different in each region? Do you find it surprising?

_Type your answer here, replacing this text._

<!-- END QUESTION -->



<!-- BEGIN QUESTION -->

**Question 9.2:** Let's look at the plot for 2008 that shows the regional economic performance in the midst of the Great Recession. The causes of the Great Recession include a combination of vulnerabilities that developed in the financial system, along with a series of triggering events that began with the bursting of the United States housing bubble in 2005–2012. As a sidenote, many empirical works suggest that housing crises usually accompany high levels of mortgage delinquencies (people default on their mortgage). 

Look at the [county-level change in mortgage delinquency figure](https://www.federalreserve.gov/images/bernanke20080505fig3.jpg) that was used in Ben Bernanke (the chairman of the Federal Reserve at that time) speech [_Mortgage Delinquencies and Foreclosures_](https://www.federalreserve.gov/newsevents/speech/bernanke20080505a.htm) at the Columbia Business School's 32nd Annual Dinner in May 2008. What is the association between this mortgage delinquency graph and the regional real GDP graph we have above? How can this result potentially inform us about the causes of the Great Recession?

_Type your answer here, replacing this text._

<!-- END QUESTION -->



We can also make some widgets!

In [ ]:
def plot_counties_widgets(industry, year):
    rgdp_county_industry = rgdp_county[rgdp_county["Description"] == industry].drop("Description", axis=1)
    rgdp_county_industry_geo = rgdp_county_industry.merge(county_geo, left_on="GeoFIPS", right_on="county_fips")
    rgdp_county_industry_geo_chg = rgdp_county_industry_geo[rgdp_county_industry_geo["year"] != 2001].copy()
    
    pct_chg_vals = rgdp_county_industry_geo[["value"]].pct_change()
    rgdp_county_industry_geo_chg["value"] = pct_chg_vals.drop(index=np.arange(0, len(pct_chg_vals), 20), axis=0)
    
    plot_counties(rgdp_county_industry_geo_chg, year, re.sub("  ", "", industry))

In [ ]:
i = widgets.Dropdown(options=rgdp_county["Description"].unique(),
                     value="All industry total", 
                     description="Industry", 
                     layout={'width': 'max-content'})

t = widgets.IntSlider(min=2002, max=2020, step=1, 
                      description="Year", 
                      layout={'width': '300px'})

interact(plot_counties_widgets, industry=i, year=t);

**Congratulations!** You're done with Econ 148 Project 1!

---
## Feedback

**Question 10:** Please fill out this short [feedback form](https://forms.gle/DzFCYCU8cVqPn2fCA) to let us know your thoughts about this project! We really appreciate your opinions and feedback! At the end of the Google form, you should see a codeword. Assign the codeword to the variable `codeword` below. 

In [ ]:
codeword = ...

In [ ]:
grader.check("q10")

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)